In [1]:
from keras.datasets import mnist
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import keras
import numpy as np
import matplotlib.pyplot as plt


In [2]:
(X_train, _),(_,_) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
X_train.shape

(60000, 28, 28)

In [4]:
 X_train  = (X_train-127.5)/127.5

 print(X_train.min())
 print(X_train.max())

-1.0
1.0


In [5]:
TOTAL_EPOCHS = 50 #number of passes
BATCH_SIZE = 256  #number of samples processed before the model is updated
HALF_BATCH = 128

NO_OF_BATCHES = int(X_train.shape[0]/BATCH_SIZE)
NOISE_DIM = 100
adam = Adam(lr = 2e-4, beta_1 = 0.5)  # lr=learning rate

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [6]:
#------------------------Generator Model : UpSampling------------------------#
generator = Sequential()
generator.add(Dense(units = 7*7*128, input_shape = (NOISE_DIM,)))
generator.add(Reshape((7,7,128)))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

#(7,7,128) -> (14,14,64)
generator.add(Conv2DTranspose(64, (3,3), strides = (2, 2), padding = 'same'))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

#(14,14,64) -> (28,28,1)
generator.add(Conv2DTranspose(1, (3,3), strides = (2, 2), padding = 'same', activation = 'tanh'))

generator.compile(loss = "binary_crossentropy", optimizer = adam)
#generator.summary()
print("Success")

Success


In [7]:
#------------------------Discriminator Model : DownSampling------------------------#
#(28,28,1) -> (14,14,64)
descriminator = Sequential()
descriminator.add(Conv2D(64, kernel_size=(3,3), strides=(2,2), padding='same', input_shape=(28,28,1)))
descriminator.add(LeakyReLU(0.2))

#(14,14,64) -> (7,7,128)
descriminator.add(Conv2D(128, kernel_size=(3,3), strides=(2,2), padding='same'))
descriminator.add(LeakyReLU(0.2))

 #(7,7,128) -> 6272
descriminator.add(Flatten())
descriminator.add( Dense(100) )
descriminator.add(LeakyReLU(0.2))

descriminator.add(Dense(1, activation='sigmoid'))
descriminator.compile(loss = "binary_crossentropy", optimizer=adam)
#descriminator.summary()

print("Success")

Success


In [8]:
## Combined Model
descriminator.trainable = False
gan_input = Input(shape = (NOISE_DIM, ))
generated_img = generator(gan_input)
gan_output = descriminator(generated_img)

# Functional
model = Model(gan_input, gan_output)
model.compile(loss = "binary_crossentropy", optimizer=adam)
#model.summary()

print("Success")

Success


In [9]:
X_train = X_train.reshape(-1,28,28,1)
X_train.shape

(60000, 28, 28, 1)

In [10]:
def display_images(samples = 25):
  noise = np.random.normal(0,1,size=(samples, NOISE_DIM))
  generated_img = generator.predict(noise)

  plt.figure(figsize = (10,10))
  
  for i in range(samples):
    plt.subplot(5, 5, i+1)
    plt.imshow(generated_img[i].reshape(28,28), cmap = 'gray')
    plt.axis("off")
  
  plt.show()

In [ ]:
## Training Loop
d_losses = []   # Descriminator loss
g_losses = []   # Generator loss

for epoch in range(TOTAL_EPOCHS):
  epoch_d_loss = 0.0
  epoch_g_loss = 0.0

  # Mini batch gradient decent
  for step in range(NO_OF_BATCHES):
    #+++++++++++++++++++++++++++++
    #Step 1 Train Descriminator
    descriminator.trainable = True

    # get the real data
    idx = np.random.randint(0, 60000, HALF_BATCH)
    real_imgs = X_train[idx]

    # get the fake data
    noise = np.random.normal(0, 1, size=(HALF_BATCH, NOISE_DIM))
    fake_imgs = generator.predict(noise)

    # Labels
    real_y = np.ones((HALF_BATCH, 1))*0.9   #One-sided label smoothing
    fake_y = np.zeros((HALF_BATCH, 1))

    # now train D
    d_loss_real = descriminator.train_on_batch(real_imgs, real_y)
    d_loss_fake = descriminator.train_on_batch(fake_imgs, fake_y)

    d_loss = 0.5*d_loss_real + 0.5*d_loss_fake
    epoch_d_loss += d_loss  ##########################

    

    #============================================
    #step 2 Train Generator (Discriminator freeze)
    descriminator.trainable = False
    noise = np.random.normal(0, 1, size=(BATCH_SIZE, NOISE_DIM))
    ground_truth_y = np.ones((BATCH_SIZE, 1))
    g_loss = model.train_on_batch(noise, ground_truth_y)
    epoch_g_loss += g_loss

    print("Epoch {} Descriminator loss {}, Generator Loss {}".format((epoch+1), epoch_d_loss/NO_OF_BATCHES, epoch_g_loss/BATCH_SIZE))
    d_losses.append(epoch_d_loss/NO_OF_BATCHES)
    g_losses.append(epoch_g_loss/NO_OF_BATCHES)

    if(epoch+1)%10 == 0:
      generator.save("generator.h5")
      display_images()